In [8]:
from langchain.chains import GraphCypherQAChain
from langchain.chat_models import ChatOpenAI
from langchain.graphs import Neo4jGraph
from langchain.callbacks import get_openai_callback
from dotenv import load_dotenv
import os
import openai


In [2]:
LLM_MODEL = 'gpt-4-32k'


In [5]:
def get_neo4j_cypher_rag_chain():
    load_dotenv(os.path.join(os.path.expanduser('~'), '.spoke_neo4j_config.env'))
    username = os.environ.get('NEO4J_USER')
    password = os.environ.get('NEO4J_PSW')
    url = os.environ.get('NEO4J_URI')
    database = os.environ.get('NEO4J_DB')

    graph = Neo4jGraph(
        url=url, 
        username=username, 
        password=password,
        database = database
    )

    load_dotenv(os.path.join(os.path.expanduser('~'), '.gpt_config.env'))
    API_KEY = os.environ.get('API_KEY')
    API_VERSION = os.environ.get('API_VERSION')
    RESOURCE_ENDPOINT = os.environ.get('RESOURCE_ENDPOINT')
    openai.api_type = "azure"
    openai.api_key = API_KEY
    openai.api_base = RESOURCE_ENDPOINT
    openai.api_version = API_VERSION
    chat_deployment_id = LLM_MODEL
    chat_model_id = chat_deployment_id
    temperature = 0
    chat_model = ChatOpenAI(openai_api_key=API_KEY, 
                            engine=chat_deployment_id, 
                            temperature=temperature)
    chain = GraphCypherQAChain.from_llm(
        chat_model, 
        graph=graph, 
        verbose=True, 
        validate_cypher=True,
        return_intermediate_steps=True
    )
    return chain

In [6]:
%%time
neo4j_rag_chain = get_neo4j_cypher_rag_chain()


WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.


CPU times: user 88.4 ms, sys: 14.2 ms, total: 103 ms
Wall time: 50.2 s


In [9]:
question = 'What are the genes associated with multiple sclerosis?'
with get_openai_callback() as cb:
    out = neo4j_rag_chain.run(query=question, return_final_only=False)
    print(out)




> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (d:Disease {name: "multiple sclerosis"})-[:ASSOCIATES_DaG]->(g:Gene)
RETURN g.name
Full Context:
[{'g.name': 'IL4'}, {'g.name': 'BTNL2'}, {'g.name': 'TGFB1'}, {'g.name': 'CBLB'}, {'g.name': 'CD6'}, {'g.name': 'OS9'}, {'g.name': 'TBX6'}, {'g.name': 'ESPN'}, {'g.name': 'IKZF3'}, {'g.name': 'AFF1'}]

> Finished chain.
The genes associated with multiple sclerosis are IL4, BTNL2, TGFB1, CBLB, CD6, OS9, TBX6, ESPN, IKZF3, and AFF1.
